#  High-temperature susceptibility data analysis

In this notebook, a contribution is downloaded from the MagIC database. Susceptibility-temperature (<span style="font-family: 'Times New Roman';">*$\chi$-T*</span>) data contained within it are extracted. The user has the option to correct for the diamagnetic signal of the measurement instrument and holder (*optional)* and to smooth the data prior to further analysis (*optional*).

The notebook is to be run cell-by-cell to allow users to select corrections/filters as needed. The code below brings in example magnetite data from the rock magnetic bestiary. Any MagIC contribution that contains <span style="font-family: 'Times New Roman';">*$\chi$-T*</span> data can be used in place of this example dataset.

### Install and import packages

In [1]:
import pmagpy.rockmag as rmag
import pmagpy.ipmag as ipmag
import pmagpy.contribution_builder as cb

from bokeh.io import output_notebook, show
output_notebook(hide_banner=True)

## Import data

We follow one of the approaches as in the `rockmag_data_unpack.ipynb` notebook to bring the MagIC data into the notebook as a Contribution.

In [2]:
#define these three parameters to match your data
magic_id = '20354'
share_key = '2f33e164-df55-4548-8d28-5b715683ae43'
dir_path = 'example_data/X-T'

result, magic_file = ipmag.download_magic_from_id(magic_id, directory=dir_path, share_key=share_key)
ipmag.unpack_magic(magic_file, dir_path, print_progress=False)
contribution = cb.Contribution(dir_path)
measurements = contribution.tables['measurements'].df

Download successful. File saved to: example_data/X-T/magic_contribution_20354.txt
1  records written to file  /Users/unimos/0000_Github/RockmagPy-notebooks/thermomagnetic_notebooks/example_data/X-T/contribution.txt
2  records written to file  /Users/unimos/0000_Github/RockmagPy-notebooks/thermomagnetic_notebooks/example_data/X-T/locations.txt
2  records written to file  /Users/unimos/0000_Github/RockmagPy-notebooks/thermomagnetic_notebooks/example_data/X-T/sites.txt
43  records written to file  /Users/unimos/0000_Github/RockmagPy-notebooks/thermomagnetic_notebooks/example_data/X-T/samples.txt
201  records written to file  /Users/unimos/0000_Github/RockmagPy-notebooks/thermomagnetic_notebooks/example_data/X-T/specimens.txt
92035  records written to file  /Users/unimos/0000_Github/RockmagPy-notebooks/thermomagnetic_notebooks/example_data/X-T/measurements.txt
-I- Using online data model
-I- Getting method codes from earthref.org
-I- Importing controlled vocabularies from https://earthref.

## The project export contains data from all the experiments

Each measurement in a MagIC measurements table has a `method_codes` value. These method codes come from a "controlled vocabulary" (https://www2.earthref.org/MagIC/method-codes). 

In the method codes used for the example contribution:
- `LP` refers to lab protocol
- `X-T` refers to susceptibility vs. temperature experiments done on the Kappabridge

In [3]:
measurements = measurements[measurements['method_codes'] == 'LP-X-T']

## Select a <span style="font-family: 'Times New Roman';">*$\chi$-T*</span> Experiment
*Creates a dropdown to select specimens with <span style="font-family: 'Times New Roman';">*$\chi$*-T</span> data.*

In [4]:
specimen, experiment = rmag.interactive_specimen_experiment_selection(measurements)

Dropdown(description='specimen:', options=('magnetite_Wright 041183-k(T)-01', 'magnetite_Wright 112978-k(T)-01…

Dropdown(description='Experiment:', options=('IRM-KappaF-LP-X-T-3407',), value='IRM-KappaF-LP-X-T-3407')

## Plot $\chi$-T data

If the `remove_holder` parameter is set to be `True`, we assume that the lowest susceptibility value during the whole measurement is the holder signal. This assumption relies on this value being after the specimen is heated above the critical temperature such that it is paramagnetic and the holder signal is dominant.

In [5]:
selected_experiment = measurements[(measurements['specimen']==specimen.value) & 
                                      (measurements['experiment']==experiment.value)].reset_index(drop=1)

fig=rmag.plot_X_T(selected_experiment, 
              temp_unit='C', 
              remove_holder=True, 
              plot_derivative=False,
              plot_inverse=False,
              return_figure=True)

## Plot $\chi$-T data with derivative

In this example, a specific experiment is specified ```'IRM-KappaF-LP-X-T-3410'``` and it is plotted with its derivative with ```plot_derivative=True```.

In [6]:
selected_experiment = measurements[measurements['experiment']=='IRM-KappaF-LP-X-T-3410'].reset_index(drop=1)

rmag.plot_X_T(selected_experiment, 
              temp_unit='C', 
              remove_holder=True, 
              plot_derivative=True,
              plot_inverse=False,)

### Estimate *Tc* using inverse susceptibility

Above its Curie temperature (*Tc*) a mineral phase will be paramagnetic. This means that the inverse susceptibility should have a linear positive slope above *Tc*. [Petrovský and Kapicka (2006)](https://doi.org/10.1029/2006JB004507) propose using the transition to linearity in inverse susceptibility  (i.e. where the Curie-Weiss linear law is obeyed) as an approach to estimate the Curie temperature.  

By setting `plot_inverse=True` within the `rmag.plot_X_T()` function, we can obtain the inverse susceptibility plot that can enable such estimates.

In [7]:
rmag.plot_X_T(selected_experiment, 
              temp_unit='C', 
              remove_holder=True, 
              plot_derivative=True,
              plot_inverse=True,)